In [13]:
import tensorflow as tf

In [14]:
URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00194/sensor_readings_24.data"
dataset = tf.keras.utils.get_file("sensor_readings_24.data",URL)

876544/875878 [==============================] - 5s 6us/step


In [3]:
!pip install -q -U tensorflow_datasets

In [4]:
import tensorflow_datasets as tfds

In [5]:
CSV_COLUMNS =  ['US1','US2','US3','US4','US5','US6','US7','US8','US9','US10','US11','US12','US13','US14','US15','US16','US17','US18','US19','US20','US21','US22','US23','US24', 'Class']

In [6]:
def train_input_fn():
  direction = tf.data.experimental.make_csv_dataset(
      dataset,
      batch_size=32,
      header=True,
      column_names=CSV_COLUMNS,
      label_name="Class")

  direction_batches = (
      direction.cache().repeat().prefetch(tf.data.AUTOTUNE))
  return direction_batches

In [7]:
def create_feature_columns():
    """Creates feature columns from inputs.

    Returns:
        Dictionary of feature columns.
    """
    feature_columns = {
        colname : tf.feature_column.numeric_column(key=colname)
           for colname in ['US1','US2','US3','US4','US5','US6','US7','US8','US9','US10','US11','US12','US13','US14','US15','US16','US17','US18','US19','US20','US21','US22','US23','US24']
    }

    return feature_columns

In [8]:
fc = create_feature_columns()
for key,val in fc.items():
    exec(key + '=val')

In [9]:
import tempfile
from tensorflow import keras

model_dir = tempfile.mkdtemp()
model = tf.estimator.DNNClassifier(
    model_dir=model_dir,
    hidden_units=[128,80,50,30,10],
    feature_columns=[US1,US2,US3,US4,US5,US6,US7,US8,US9,US10,US11,US12,US13,US14,US15,US16,US17,US18,US19,US20,US21,US22,US23,US24],
    n_classes=4,
    label_vocabulary=['Move-Forward','Slight-Right-Turn','Sharp-Right-Turn', 'Slight-Left-Turn'],
    activation_fn='relu',
    optimizer=keras.optimizers.Adam(learning_rate=0.005)
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6vti08ko', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
model = model.train(input_fn=train_input_fn, steps=12000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp6vti08ko/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.4008635, step = 0
INFO:tensorflow:global_step/sec: 147.93
INFO:tensorflow:loss = 0.68076897, step = 100 (0.677 sec)
INFO:tensorflow:global_step/sec: 124.366
INFO:tensorflow:loss = 0.65019697, step = 200 (0.810 sec)
INFO:tensorflow:global_step/sec: 147.82
INFO:tensorflow:loss = 0.6599213, step = 300 (0.671 sec)
INFO:tensorflow:global_step/sec: 125.681
INFO:tenso

In [11]:
result = model.evaluate(train_input_fn, steps=10)

for key, value in result.items():
  print(key, ":", value)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-21T14:07:13Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6vti08ko/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Inference Time : 1.12819s
INFO:tensorflow:Finished evaluation at 2021-01-21-14:07:15
INFO:tensorflow:Saving dict for global step 12000: accuracy = 0.959375, average_loss = 0.095914975, global_step = 12000, loss = 0.095914975
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12000: /tmp/tmp6vti08ko/

In [12]:
for pred in model.predict(train_input_fn):
  for key, value in pred.items():
    print(key, ":", value)
  break

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6vti08ko/model.ckpt-12000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
logits : [  7.2776856  -1.052438   -7.234323  -17.232327 ]
probabilities : [9.9975842e-01 2.4108392e-04 4.9820699e-07 2.2663775e-11]
class_ids : [0]
classes : [b'Move-Forward']
all_class_ids : [0 1 2 3]
all_classes : [b'Move-Forward' b'Slight-Right-Turn' b'Sharp-Right-Turn'
 b'Slight-Left-Turn']
